## Bitcoin

In [2]:
import pandas as pd, tweepy

In [1]:
!snscrape twitter-search "#Bitcoin since:2020-10-01 until:2020-11-01" > bitcoin_tweets.txt

2020-11-02 19:36:58.764  WARNING  snscrape.base  Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23Bitcoin+since%3A2020-10-01+until%3A2020-11-01&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaOgLCdkL_k0CQWgoC1_fzXkdskEjUAFQAlABEVh5FUFYCJehgEVVNFUhUAFQAVARXoGhUAAA%3D%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed 

API Keys from Twitter Developer Account

In [ ]:
consumer_key = "QQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ" 
consumer_secret = "UUUUUUUUUUUUUUUUUUUUUUUUUUU" 
access_token = "EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE" 
access_token_secret = "NNNNNNNNNNNNNNNNNNNNNNNNN"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

 Open .txt or .json using pandas.read_json or pandas.read_csv

In [3]:
tweet_url = pd.read_csv("bitcoin_tweets.txt", index_col= None, header = None, names = ["links"])
tweet_url.head()

,links
0,https://twitter.com/JoaoDMendes/status/1322689...
1,https://twitter.com/Z06Z07/status/132268980142...
2,https://twitter.com/PoisonI96460421/status/132...
3,https://twitter.com/CryptoWatchBot/status/1322...
4,https://twitter.com/planethardmoney/status/132...


 Extract tweet_id using .split function

In [9]:
af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)
tweet_url.head()

,links,id
0,https://twitter.com/JoaoDMendes/status/1322689...,1322689810381840385
1,https://twitter.com/Z06Z07/status/132268980142...,1322689801422696450
2,https://twitter.com/PoisonI96460421/status/132...,1322689799975653376
3,https://twitter.com/CryptoWatchBot/status/1322...,1322689779755032577
4,https://twitter.com/planethardmoney/status/132...,1322689752978624513


Convert tweet_url Series into list

In [10]:
ids = tweet_url['id'].tolist()

Issue looping through all ids, the API is giving an error. Therefore, process the ids by batch or chunks

In [11]:
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

Create function that would extract all elements that we need from the tweet (username, date and the tweet). Add hashtags???

In [12]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at}
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("sherlock_tweets.csv", mode="a")

Create another for loop to loop into our batches while processing 50 entries every loop. (That felt like a tongue twister)

In [13]:
for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)

NameError: name 'api' is not defined